In [344]:
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
from meteostat import Point, Daily, Hourly
import numpy as np
import pandas as pd
import geopy.distance
import glob

In [8]:
data = pd.read_csv("../processed/processed-shutoffs.csv")
data.head()

,circuit_name,deenergize_time,restoration_time,time_out_min,key_communities,hftd_tier,total_affected,residential_affected,zip_code,longitude,latitude,substn_present,total_population
0,APPLE HILL-1103,2018-10-14 21:00:00,2018-10-15 16:29:00.000000000,1169.0,"Camino, Placerville",3.0,1809.0,1488.0,95709,-120.677165,38.739683,True,5634.0
1,APPLE HILL-1104,2018-10-14 21:17:00,2018-10-15 18:37:00.000000000,1280.0,"Camino, Placerville, Pollock Pines,",3.0,2261.0,2043.0,95709,-120.677165,38.739683,True,5634.0
2,APPLE HILL-2102,2018-10-14 21:05:00,2018-10-16 18:04:00.000000000,2699.0,"Camino, Grizzly Flats, Mount Aukum, Placervill...",3.0,4489.0,4013.0,95709,-120.677165,38.739683,True,5634.0
3,BONNIE NOOK-1102,2018-10-14 21:54:00,2018-10-15 18:04:00.000000000,1210.0,Alta,3.0,533.0,454.0,95701,-120.809292,39.214708,True,1064.0
4,CALISTOGA-1101,2018-10-14 20:34:00,2018-10-15 21:27:00.000000000,1493.0,Calistoga,3.0,1596.0,1216.0,94515,-122.578610,38.592088,True,7525.0


In [21]:
data["deenergize_time"] = pd.to_datetime(data["deenergize_time"])
data["restoration_time"] = pd.to_datetime(data["restoration_time"])
data["deenergize_date"] = data["deenergize_time"].dt.date
data["restoration_date"] = data["restoration_time"].dt.date


In [555]:
missing = {}
full_data = {}
weather_present = {}
data_missing = {}
daily = {}
daily_data_missing = {}
for i in range(data.shape[0]):
    point_1 = Point(data.loc[i, "latitude"], data.loc[i, "longitude"])
    start = pd.to_datetime(data.loc[i, "deenergize_date"]) - timedelta(days=5)
    end = pd.to_datetime(data.loc[i, "deenergize_date"])
    # weather = Hourly(point_1, start, end)
    weather = Daily(point_1, start, end)
    weather = weather.fetch()
    # if weather.shape[0] == 0:
    #     weather = Daily(point_1, start, end)
    #     weather = weather.fetch()
    if weather.shape[0] == 0:
        missing[i] = data.loc[i, ["circuit_name", "deenergize_time", "latitude", "longitude"]]
    #     daily[i] = weather
    elif weather.shape[0] >= 5:
        if weather.shape[0] > 5:
            weather = weather.loc[start.date(): end.date() - timedelta(days=1), :]
            assert weather.shape[0] == 5
        if sum(np.sum(weather[["tmin", "tmax", "wspd"]].isna(), axis=1)) == 0:
            full_data[i] = data.loc[i, ["circuit_name", "deenergize_time", "latitude", "longitude"]]
            weather["circuit_name"] = data.loc[i, "circuit_name"]
            weather["deenergize_time"] = data.loc[i, "deenergize_time"]
            weather["source_circuit_name"] = data.loc[i, "circuit_name"]
            weather["source_deenergize_time"] = data.loc[i, "deenergize_time"]
            weather["approximated"] = False
            daily[i] = weather
        else:
            data_missing[i] = data.loc[i, ["circuit_name",
                                "deenergize_time", "latitude", "longitude"]]
            weather["circuit_name"] = data.loc[i, "circuit_name"]
            weather["deenergize_time"] = data.loc[i, "deenergize_time"]
            weather["source_circuit_name"] = data.loc[i, "circuit_name"]
            weather["source_deenergize_time"] = data.loc[i, "deenergize_time"]
            weather["approximated"] = False
            daily_data_missing[i] = weather
    else:
        weather_present[i] = data.loc[i, ["circuit_name", "deenergize_time", "latitude", "longitude"]]
        

In [556]:
full_data = pd.DataFrame(full_data).T
full_data.to_csv("full_weather.csv")
missing = pd.DataFrame(missing).T
missing.to_csv("missing_weather.csv")
weather_present = pd.DataFrame(weather_present).T
weather_present.to_csv("some_missing_weather.csv")
data_missing = pd.DataFrame(data_missing).T
data_missing.to_csv("data_missing.csv")


In [427]:
for i in list(daily.keys()):
    shutoff = data.iloc[i, :]
    filename = f"raw/daily/weather_{i}_{shutoff['circuit_name']}_{shutoff['deenergize_date']}.csv"
    weather = daily[i]
    assert os.path.isfile(filename) == False
    weather.to_csv(filename)
    

In [373]:
# for i in list(hourly.keys()):
#     shutoff = data.iloc[i, :]
#     filename = f"raw/hourly/weather_{shutoff['circuit_name']}_{shutoff['deenergize_date']}_hourly.csv"
#     weather = hourly[i]
#     weather.to_csv(filename)


In [483]:
# Replace stations that are entirely missing weather data with a copy
# of their nearest neighbor's weather

knn_missing = {}
to_drop = {}
for i in missing.index:
    de_time = missing.loc[i, "deenergize_time"].date()
    dist = full_data[full_data["deenergize_time"].dt.date == de_time].apply(lambda row: geopy.distance.geodesic(
        (row["latitude"], row["longitude"]),
        (missing.loc[i, "latitude"],
         missing.loc[i, "longitude"])).miles,
        axis=1)
    if len(dist) == 0:
        to_drop[i] = missing.loc[i]
        continue
    min_idx = dist.idxmin()
    dist = min(dist)
    if dist > 60:
        to_drop[i] = missing.loc[i]
        continue
    knn_missing[i] = {
        "distance": dist,
        "min_idx": min_idx,
        "closest_circuit_name": full_data.loc[min_idx].loc["circuit_name"],
        "closest_deenergize_time": full_data.loc[min_idx].loc["deenergize_time"],
        "closest_latitude": full_data.loc[min_idx].loc["latitude"],
        "closest_longitude": full_data.loc[min_idx].loc["longitude"]
    }


In [506]:
# Join to get rest for information
knn_missing = pd.DataFrame(knn_missing).T.join(missing)

# Create copy of weather from nearest neighbor, write to disk
for i in knn_missing.index:
    file_name = glob.glob(
        f"raw/daily/weather_{knn_missing['min_idx'].loc[i]}_{knn_missing['closest_circuit_name'].loc[i]}_{str(knn_missing['closest_deenergize_time'].loc[i].date())}.csv")
    assert len(file_name) == 1
    curr_weather = pd.read_csv(file_name[0])
    curr_weather["circuit_name"] = knn_missing['circuit_name'].loc[i]
    curr_weather["deenergize_time"] = knn_missing['deenergize_time'].loc[i]
    curr_weather["source_circuit_name"] = knn_missing['closest_circuit_name'].loc[i]
    curr_weather["source_deenergize_time"] = knn_missing['closest_deenergize_time'].loc[i]
    curr_weather["approximated"] = True
    curr_weather.to_csv(
        f"raw/daily/weather_{i}_{knn_missing['circuit_name'].loc[i]}_{str(knn_missing['deenergize_time'].loc[i].date())}.csv")


In [531]:
# Replace stations that are missing some rows of weather with:
# a copy of their nearest neighbor's weather
    
knn_some_missing = {}
for i in weather_present.index:
    de_time = weather_present.loc[i, "deenergize_time"].date()
    dist = full_data[full_data["deenergize_time"].dt.date == de_time].apply(lambda row: geopy.distance.geodesic(
        (row["latitude"], row["longitude"]),
        (weather_present.loc[i, "latitude"],
         weather_present.loc[i, "longitude"])).miles,
        axis=1)
    if len(dist) == 0:
        to_drop[i] = weather_present.loc[i]
        continue
    min_idx = dist.idxmin()
    dist = min(dist)
    if dist > 60:
        to_drop[i] = weather_present.loc[i]
        continue
    knn_some_missing[i] = {
        "distance": dist,
        "min_idx": min_idx,
        "closest_circuit_name": full_data.loc[min_idx].loc["circuit_name"],
        "closest_deenergize_time": full_data.loc[min_idx].loc["deenergize_time"],
        "closest_latitude": full_data.loc[min_idx].loc["latitude"],
        "closest_longitude": full_data.loc[min_idx].loc["longitude"]
    }


In [534]:
# Join to get rest for information
knn_some_missing = pd.DataFrame(knn_some_missing).T.join(weather_present)


In [536]:
# Create copy of weather from nearest neighbor, write to disk
for i in knn_some_missing.index:
    file_name = glob.glob(
        f"raw/daily/weather_{knn_some_missing['min_idx'].loc[i]}_{knn_some_missing['closest_circuit_name'].loc[i]}_{str(knn_some_missing['closest_deenergize_time'].loc[i].date())}.csv")
    assert len(file_name) == 1
    curr_weather = pd.read_csv(file_name[0])
    curr_weather["circuit_name"] = knn_some_missing['circuit_name'].loc[i]
    curr_weather["deenergize_time"] = knn_some_missing['deenergize_time'].loc[i]
    curr_weather["source_circuit_name"] = knn_some_missing['closest_circuit_name'].loc[i]
    curr_weather["source_deenergize_time"] = knn_some_missing['closest_deenergize_time'].loc[i]
    curr_weather["approximated"] = True
    curr_weather.to_csv(
        f"raw/daily/weather_{i}_{knn_some_missing['circuit_name'].loc[i]}_{str(knn_some_missing['deenergize_time'].loc[i].date())}.csv")


In [538]:
pd.DataFrame(to_drop).T.to_csv("to_drop.csv")

In [545]:
tmin_missing

,circuit_name,deenergize_time,latitude,longitude
190,BOLINAS1101,2019-10-09 00:45:00,37.941212,-122.706149
195,BRUNSWICK1102,2019-10-09 03:13:00,39.23105,-121.035026
196,BRUNSWICK1103,2019-10-09 03:13:00,39.23105,-121.035026
197,BRUNSWICK1104,2019-10-09 03:13:00,39.23105,-121.035026
198,BRUNSWICK1105,2019-10-09 03:11:00,39.23105,-121.035026
...,...,...,...,...
1787,WEIMAR,2020-10-25 16:18:00,39.035963,-120.987984
1788,WEIMAR,2020-10-25 16:19:00,39.035963,-120.987984
2044,PINECREST0401,2019-10-09 15:31:00,38.3411,-119.8327
2101,WINDSOR1101,2019-10-26 19:50:00,38.5468,-122.8197


In [611]:
data_missing

,circuit_name,deenergize_time,latitude,longitude
31,BANGOR1101,2019-06-08 20:45:00,39.387674,-121.403176
32,BUTTE1105,2019-06-08 20:55:00,39.712206,-121.813357
33,CLARK ROAD1101,2019-06-08 21:12:00,39.654521,-121.635508
35,ORO FINO1101,2019-06-08 21:04:00,39.871563,-121.610785
36,OROVILLE0403,2019-06-08 21:19:00,39.503998,-121.560155
...,...,...,...,...
2101,WINDSOR1101,2019-10-26 19:50:00,38.5468,-122.8197
2102,WINDSOR1102,2019-10-26 19:50:00,38.5468,-122.8197
2104,PINECREST,2020-09-08 00:21:00,38.3411,-119.8327
2125,TAR FLAT,2020-10-25 16:03:00,37.9768,-120.3666


In [613]:
# Replace stations that are missing tmin with:
# a copy of their nearest neighbor's tmin
knn_data_missing = {}
for i in data_missing.index:
    de_time = data_missing.loc[i, "deenergize_time"].date()
    dist = full_data[full_data["deenergize_time"].dt.date == de_time].apply(lambda row: geopy.distance.geodesic(
        (row["latitude"], row["longitude"]),
        (data_missing.loc[i, "latitude"],
         data_missing.loc[i, "longitude"])).miles,
        axis=1)
    if len(dist) == 0:
        to_drop[i] = data_missing.loc[i]
        continue
    min_idx = dist.idxmin()
    dist = min(dist)
    if dist > 60:
        to_drop[i] = data_missing.loc[i]
        continue
    knn_data_missing[i] = {
        "distance": dist,
        "min_idx": min_idx,
        "closest_circuit_name": full_data.loc[min_idx].loc["circuit_name"],
        "closest_deenergize_time": full_data.loc[min_idx].loc["deenergize_time"],
        "closest_latitude": full_data.loc[min_idx].loc["latitude"],
        "closest_longitude": full_data.loc[min_idx].loc["longitude"]
    }
    daily_data_missing[i].loc[:, ["tmin", "tmax", "wspd"]] = daily_data_missing[i].loc[:, ["tmin", "tmax", "wspd"]].reset_index().fillna(
        daily[min_idx][["tmin", "tmax", "wspd"]].reset_index()).set_index("time")
    daily_data_missing[i]["approximated"] = True
    daily_data_missing[i].to_csv(
        f"raw/daily/weather_{i}_{data_missing['circuit_name'].loc[i]}_{str(data_missing['deenergize_time'].loc[i].date())}.csv")


In [621]:
# pd.DataFrame(to_drop).T.to_csv("to_drop.csv")

In [615]:
len(to_drop)

105

In [617]:
len(os.listdir('raw/daily/'))

2032

In [618]:
data.shape[0]

2137